In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
from time import strftime
from time import sleep
import warnings
warnings.filterwarnings("ignore")
import json
import unicodedata
import re

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import acquire
from prepare import *

In [2]:
data = pd.read_json('data.json')

In [3]:
data = data.dropna()

In [4]:
data

,repo,language,readme_contents
1,onevcat/Kingfisher,Swift,"<p align=""center"">\n<img src=""https://raw.gith..."
3,tailwindlabs/tailwindcss,JavaScript,"<p>\n <a href=""https://tailwindcss.com/"" ta..."
5,github/fetch,JavaScript,# window.fetch polyfill\n\nThe `fetch()` funct...
7,ianstormtaylor/slate,TypeScript,"<p align=""center"">\n <a href=""#""><img src=""./..."
8,Kong/insomnia,JavaScript,# Insomnia REST Client\n\n[![Slack Channel](ht...
...,...,...,...
555,tornadoweb/tornado,Python,Tornado Web Server\n==================\n\n.. i...
556,emscripten-core/emscripten,C,![emscripten logo](media/switch_logo.png)\n\nM...
557,gulpjs/gulp,JavaScript,"<p align=""center"">\n <a href=""https://gulpjs...."
558,alibaba/weex,C++,# Weex\n\nA framework for building Mobile cros...


In [5]:
data= data[data['language'].map(data['language'].value_counts()) >= 30]

In [6]:
data = data.reset_index()

In [7]:
data = prep_readme_data(df= data, column = 'readme_contents', extra_words=['https','p','com','http','href'], exclude_words=[])

In [8]:
data.head()

,readme_contents,repo,language,clean,stemmed,lemmatized
0,"<p>\n <a href=""https://tailwindcss.com/"" ta...",tailwindlabs/tailwindcss,JavaScript,hrefhttpstailwindcsscom target_blank img altta...,hrefhttpstailwindcsscom target_blank img altta...,hrefhttpstailwindcsscom target_blank img altta...
1,# window.fetch polyfill\n\nThe `fetch()` funct...,github/fetch,JavaScript,windowfetch polyfill fetch function promisebas...,windowfetch polyfil fetch function promisebas ...,windowfetch polyfill fetch function promisebas...
2,"<p align=""center"">\n <a href=""#""><img src=""./...",ianstormtaylor/slate,TypeScript,aligncenter hrefimg srcdocsimagesbannerpng ali...,aligncent hrefimg srcdocsimagesbannerpng align...,aligncenter hrefimg srcdocsimagesbannerpng ali...
3,# Insomnia REST Client\n\n[![Slack Channel](ht...,Kong/insomnia,JavaScript,insomnia rest client slack channelhttpschatins...,insomnia rest client slack channelhttpschatins...,insomnia rest client slack channelhttpschatins...
4,![](https://user-images.githubusercontent.com/...,CymChad/BaseRecyclerViewAdapterHelper,Java,httpsuserimagesgithubusercontentcom76982093319...,httpsuserimagesgithubusercontentcom76982093319...,httpsuserimagesgithubusercontentcom76982093319...


In [15]:
data[0:1].clean

0    hrefhttpstailwindcsscom target_blank img altta...
Name: clean, dtype: object

In [9]:
data = data.drop(columns = ['readme_contents'])

In [10]:
data.language.value_counts()

JavaScript    141
Python         59
Java           46
TypeScript     45
Go             36
C++            30
Name: language, dtype: int64

In [11]:
js_df = data[data.language == 'JavaScript']
python_df = data[data.language == 'Python']
java_df = data[data.language == 'Java']
ts_df = data[data.language == 'TypeScript']
go_df = data[data.language == 'Go']
c_df = data[data.language == 'C++']

In [20]:
js_df['message_length'] = js_df.clean.apply(len)

js_df['word_count'] = js_df.clean.apply(str.split).apply(len)

In [21]:
js_df

,repo,language,clean,stemmed,lemmatized,word_count,message_length
0,tailwindlabs/tailwindcss,JavaScript,hrefhttpstailwindcsscom target_blank img altta...,hrefhttpstailwindcsscom target_blank img altta...,hrefhttpstailwindcsscom target_blank img altta...,72,1171
1,github/fetch,JavaScript,windowfetch polyfill fetch function promisebas...,windowfetch polyfil fetch function promisebas ...,windowfetch polyfill fetch function promisebas...,788,7862
3,Kong/insomnia,JavaScript,insomnia rest client slack channelhttpschatins...,insomnia rest client slack channelhttpschatins...,insomnia rest client slack channelhttpschatins...,321,3283
5,SheetJS/sheetjs,JavaScript,sheetjshttpssheetjscom parser writer various s...,sheetjshttpssheetjscom parser writer variou sp...,sheetjshttpssheetjscom parser writer various s...,9602,74603
8,angular/material,JavaScript,material design angularjs apps npm versionhttp...,materi design angularj app npm versionhttpsbad...,material design angularjs apps npm versionhttp...,686,7287
...,...,...,...,...,...,...,...
342,mochajs/mocha,JavaScript,aligncenter img srchttpscldupcomxfvfxoioausvg ...,aligncent img srchttpscldupcomxfvfxoioausvg al...,aligncenter img srchttpscldupcomxfvfxoioausvg ...,193,3698
343,less/less.js,JavaScript,aligncenterimg srchttplesscssorgpublicimgless_...,aligncenterimg srchttplesscssorgpublicimgless_...,aligncenterimg srchttplesscssorgpublicimgless_...,144,2082
346,jashkenas/backbone,JavaScript,____ __ __ _ __ __ ___ ____ ___ ___ __ _ ____ ...,____ __ __ _ __ __ ___ ____ ___ ___ __ _ ____ ...,____ __ __ _ __ __ ___ ____ ___ ___ __ _ ____ ...,115,1037
348,ryanmcdermott/clean-code-javascript,JavaScript,cleancodejavascript table contents 1 introduct...,cleancodejavascript tabl content 1 introductio...,cleancodejavascript table content 1 introducti...,4082,38482


'hrefhttpstailwindcsscom target_blank img alttailwind css width350 srchttpsrefactoringuinyc3cdndigitaloceanspacescomtailwindlogostickersvg abr utilityfirst css framework rapidly building custom user interfaces hrefhttpsgithubcomtailwindlabstailwindcssactionsimg srchttpsimgshieldsiogithubworkflowstatustailwindlabstailwindcssnodejs20ci altbuild statusa hrefhttpswwwnpmjscompackagetailwindcssimg srchttpsimgshieldsionpmdttailwindcsssvg alttotal downloadsa hrefhttpsgithubcomtailwindcsstailwindcssreleasesimg srchttpsimgshieldsionpmvtailwindcsssvg altlatest releasea hrefhttpsgithubcomtailwindcsstailwindcssblobmasterlicenseimg srchttpsimgshieldsionpmltailwindcsssvg altlicensea documentation full documentation visit tailwindcsscomhttpstailwindcsscom community help discussion best practices conversation would benefit searchable discuss tailwind css githubhttpsgithubcomtailwindcsstailwindcssdiscussions casual chitchat others using framework join tailwind css discord serverhttpsdiscordgg7nf8gne con